***小结***  
学习链接https://www.cnblogs.com/alex3714/articles/5765046.html

* 凡是可作用于for循环的对象都是Iterable类型；

* 凡是可作用于next()函数的对象都是Iterator类型，它们表示一个惰性计算的序列；

* 集合数据类型如list、dict、str等是Iterable但不是Iterator，不过可以通过iter()函数获得一个Iterator对象。

* Python的for循环本质上就是通过不断调用next()函数实现的

你可能会问，为什么list、dict、str等数据类型不是Iterator？

这是因为Python的Iterator对象表示的是一个数据流，Iterator对象可以被next()函数调用并不断返回下一个数据，直到没有数据时抛出StopIteration错误。可以把这个数据流看做是一个有序序列，但我们却不能提前知道序列的长度，只能不断通过next()函数实现按需计算下一个数据，所以Iterator的计算是惰性的，只有在需要返回下一个数据时它才会计算。

Iterator甚至可以表示一个无限大的数据流，例如全体自然数。而使用list是永远不可能存储全体自然数的。

# 生成器generator （yield方法）

通过列表生成式，我们可以直接创建一个列表。但是，受到内存限制，列表容量肯定是有限的。而且，创建一个包含100万个元素的列表，不仅占用很大的存储空间，如果我们仅仅需要访问前面几个元素，那后面绝大多数元素占用的空间都白白浪费了。

所以，如果列表元素可以按照某种算法推算出来，那我们是否可以在循环的过程中不断推算出后续的元素呢？这样就不必创建完整的list，从而节省大量的空间。在Python中，这种一边循环一边计算的机制，称为生成器：generator。
+ 只有在调用的时候才会生成相应的数据
+ 只能通过__next__()一个一个往后取值，但是可以用for循环来迭代数据
+ 如果for循环无法实现时，可以用函数实现

In [10]:
a=[i*2 for i in range(10)] #list ，列表生成式
print(a,type(a))

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18] <class 'list'>


In [11]:
b=(i*2 for i in range(10))#generator，生成器
print(b,type(b))

<generator object <genexpr> at 0x0000026BBD1AC480> <class 'generator'>


In [13]:
a[3]

TypeError: 'generator' object is not subscriptable

In [20]:
b[3] #TypeError: 'generator' object is not subscriptable

TypeError: 'generator' object is not subscriptable

In [17]:
b.__next__()

0

In [18]:
b.__next__()

2

In [19]:
b.__next__()

4

In [78]:
#斐波那契数列（Fibonacci）
def fib(max):
    n,a,b=0,0,1
    while n<max:
        a,b=b,a+b
        print(b)
        n=n+1
    return "done" 
#注意：a,b=a,a+b 相当于t=(b,a+b) a=t[0],b=t[1]；而不是a=a,b=a+b()

In [79]:
fib(10)

1
2
3
5
8
13
21
34
55
89


'done'

# yield将该函数变成生成器（generator）
+ yield中断并保存函数当前执行状态，并支持按需返回

In [91]:
def fib(max):
    n,a,b=0,0,1
    while n<max:
        a,b=b,a+b
        yield b
        n=n+1
    return "--done--" 

In [92]:
f=fib(10)
print(f,type(f))

<generator object fib at 0x0000026BBD25CB10> <class 'generator'>


In [93]:
f.__next__()

1

In [94]:
f.__next__()

2

In [95]:
f.__next__() #取不到时，会抛异常

3

In [96]:
print("=========start loop==============")
for i in f:
    print(i) #接着next执行结果，继续打印出来

=========start loop==============
5
8
13
21
34
55
89


# 抓住异常错误值
try:  
except:

In [97]:
g=fib(6)
while True:
    try:
        x=next(g)
        print('g',x)
    except StopIteration as e:
        print("Generator return value:",e.value)
        break

g 1
g 2
g 3
g 5
g 8
g 13
Generator return value: --done--


# 通过yield实现在单线程的情况下实现并发运算的效果

In [113]:
import time
def consumer(name):  #消费者
    print("%s 准备吃包子啦!" %name)
    while True:
       baozi = yield #无返回值
       print("包子[%s]来了,被[%s]吃了!" %(baozi,name))

c=consumer("chenronghua")#直接变成generator，但普通函数是直接调用。
c.__next__()  #初始化
b1="韭菜馅"
c.send(b1) #send作用是唤醒generator并传值给yield，然后yield再将值传给baozi。next只是唤醒generator

chenronghua 准备吃包子啦!
包子[韭菜馅]来了,被[chenronghua]吃了!


In [114]:
import time
def consumer(name): #生产者
    print("%s 准备吃包子啦!" %name)
    while True:
       baozi = yield #无返回值
       print("包子[%s]来了,被[%s]吃了!" %(baozi,name))
        
def producer(name):
    c = consumer('A')
    c2 = consumer('B')
    c.__next__()
    c2.__next__()
    print("老子开始准备做包子啦!")
    for i in range(5):
        time.sleep(1)
        print("做了2个包子!")
        c.send(i)
        c2.send(i)

producer("alex")

A 准备吃包子啦!
B 准备吃包子啦!
老子开始准备做包子啦!
做了2个包子!
包子[0]来了,被[A]吃了!
包子[0]来了,被[B]吃了!
做了2个包子!
包子[1]来了,被[A]吃了!
包子[1]来了,被[B]吃了!
做了2个包子!
包子[2]来了,被[A]吃了!
包子[2]来了,被[B]吃了!
做了2个包子!
包子[3]来了,被[A]吃了!
包子[3]来了,被[B]吃了!
做了2个包子!
包子[4]来了,被[A]吃了!
包子[4]来了,被[B]吃了!


# 迭代器



这些可以直接作用于for循环的对象统称为***可迭代对象***：**<font color=#FF0000> Iterable**。  
可以使用isinstance()判断一个对象是否是Iterable对象：  
可以直接作用于for循环的数据类型有以下几种：

+ 一类是集合数据类型，如list、tuple、dict、set、str等；

+ 一类是generator，包括生成器和带yield的generator function。

而生成器不但可以作用于for循环，还可以被next()函数不断调用并返回下一个值，直到最后抛出StopIteration错误表示无法继续返回下一个值了。
<font color=#FF0000>可以被next()函数调用并不断返回下一个值的对象称为***迭代器***：**<font color=#FF0000> Iterator**。

In [139]:
from collections import Iterable
from collections import Iterator

d:\software\python\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  


In [127]:
isinstance('abc',Iterable)

True

In [128]:
isinstance([1,2,3],Iterable)

True

In [129]:
isinstance([i for i in range(5)],Iterable)

True

In [130]:
isinstance((i for i in range(5)),Iterable)

True

In [131]:
isinstance(12345,Iterable) #整数

False

# 可以使用isinstance()判断一个对象是否是Iterator对象
+ isinstance((a,Iterator) 

In [133]:
a=[1,2,3]
dir(a) #dir可以查看对象的可使用方法

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [140]:

isinstance([i for i in range(5)],Iterable) # 《Iterable可迭代对象》和《Iterator迭代器对象》

True

In [144]:
isinstance([i for i in range(5)],Iterator)

False

In [142]:
isinstance((i for i in range(5)),Iterable)

True

In [145]:
isinstance((i for i in range(5)),Iterator) #迭代器对象一定是可迭代对象

True

# 可以使用iter()方法将list、dict、str变成迭代器
* iter(a)

In [146]:
iter(a) #可以将 list变成一个迭代器

In [149]:
iter('abc')

In [163]:
iter({'n':1,'m':2})

# 将字符串转成可执行代码
exec(code)可以直接执行字符串代码
![image.png](attachment:image.png)